In [48]:
%%time 
import pyspark
import string
from pecab import PeCab
import pandas as pd
import numpy as np
from collections import Counter
from pyspark.sql.functions import pandas_udf, monotonically_increasing_id, regexp_replace, sum, asc, \
                                   arrays_zip, col, explode, sort_array, struct, lit, PandasUDFType, coalesce, \
                                    concat

from pyspark.sql.types import StructType, StructField, StringType, MapType, ArrayType, IntegerType
import re
import subprocess


# local[*] : 모든 코어를 사용하겠다, local[4] : 4개의 코어를 사용하겠다.

spark = SparkSession.builder\
        .master("local[3]")\
        .appName("WordCount")\
        .config("spark.driver.memory", "2g")\
        .getOrCreate()
sc=spark.sparkContext

data_name= "test_csv"
origin_df = spark.read.options(header='True', inferSchema='True', delimiter='|',quote='"', multiLine="true").option("encoding", "UTF-8").csv(f"file:///home/j8a507/cluster/news/{data_name}.csv")

@pandas_udf(StringType())
def rm_space_in_quote_udf(x: pd.Series) -> pd.Series:
    def rm_space_in_quote(string):
        encoded_string = string.encode('utf-8')
        decoded_string = encoded_string.decode('utf-8')
        no_space_string = re.sub(r"'[^']*'", lambda m: m.group().replace(" ", ""), decoded_string)
        return no_space_string

    return x.apply(rm_space_in_quote)

df =  origin_df.withColumn("total_content", concat(col("content"), col("summary"), col("title")))\
    .select(col("idol"),col("total_content"))

df = df.withColumn("total_content", rm_space_in_quote_udf(df["total_content"]))
user_dict= {"뉴진스","트와이스"}


# 사용자 정의사전 만들기
@pandas_udf(ArrayType(StringType()))
def make_user_dict_udf(x: pd.Series) -> pd.Series:
    def make_user_dict(string):
        encoded_string = string.encode('utf-8')
        decoded_string = encoded_string.decode('utf-8')
        substrings = re.findall(r"'(.*?)'", decoded_string)
        return substrings
    return x.apply(make_user_dict)

df_dict = df.select("total_content")
df_dict= df_dict.withColumn("total_content", make_user_dict_udf(df["total_content"]))
# df_dict.collect()
for w in df_dict.toLocalIterator():
    user_dict.update(w["total_content"])

pecab = PeCab(user_dict=user_dict)

@pandas_udf(StringType())
def clean_str_udf(x: pd.Series) -> pd.Series:
    def clean_str(x):
        punc = string.punctuation
        for ch in punc:
            x = x.replace(ch, '')
        return x
    return x.apply(clean_str)

df = df.withColumn("total_content", clean_str_udf(df["total_content"]))

def set_words(words):
    counter = Counter(words)
    return counter.most_common(10)

def nouns_row(x):
    content = x[1]
    nouns = pecab.nouns(content)
    return (x[0], nouns)

def set_words_row(x):
    keywords = x[1]
    counter = Counter(keywords)
    return (x[0], counter.most_common(10))

words_rdd = df.rdd
words_rdd = words_rdd.map(lambda x : nouns_row(x))
words_rdd = words_rdd.map(lambda x: set_words_row(x))

total_rdd = words_rdd.flatMap(lambda x: x[1]).reduceByKey(lambda x, y: x + y).sortBy(lambda x: x[1], ascending=False)
words_rdd = words_rdd.zipWithIndex()

words_rdd.saveAsTextFile("output/keyword")
total_rdd.saveAsTextFile("output/total")
# Replace "/path/to/partitioned/data" with the path to your partitioned data in HDFS
hdfs_path_keyword = "output/keyword"
hdfs_path_total = "output/total"
hdfs_path_count = "output/count"


# Replace "/path/to/local/directory" with the path to your local directory
local_path_keyword = "file:///home/j8a507/cluster/news/keyword/idol_keywords.txt"
local_path_total = "file:///home/j8a507/cluster/news/total/total_keywords.txt"
local_path_count = "file:///home/j8a507/cluster/news/count/idol_count.txt"

origin_df.groupBy("idol").count().write.csv("output/count")
subprocess.check_call(["hdfs", "dfs", "-getmerge", hdfs_path_count, local_path_count])
subprocess.check_call(["hdfs", "dfs", "-getmerge", hdfs_path_keyword, local_path_keyword])
subprocess.check_call(["hdfs", "dfs", "-getmerge", hdfs_path_total, local_path_total])


CPU times: user 232 ms, sys: 293 ms, total: 526 ms
Wall time: 27.3 s


0

In [ ]:
%%time 
import pyspark
import string
from pecab import PeCab
import pandas as pd
import numpy as np
from collections import Counter
from pyspark.sql.functions import pandas_udf, monotonically_increasing_id, regexp_replace, sum, asc, \
                                   arrays_zip, col, explode, sort_array, struct, lit, PandasUDFType, coalesce 
from pyspark.sql.types import StructType, StructField, StringType, MapType, ArrayType
import re
import subprocess


# local[*] : 모든 코어를 사용하겠다, local[4] : 4개의 코어를 사용하겠다.

spark = SparkSession.builder\
        .master("local[3]")\
        .appName("WordCount")\
        .config("spark.driver.memory", "2g")\
        .getOrCreate()
sc=spark.sparkContext

data_name= "test_csv"
origin_df = spark.read.options(header='True', inferSchema='True', delimiter='|',quote='"', multiLine="true").option("encoding", "UTF-8").csv(f"file:///home/j8a507/cluster/news/{data_name}.csv")

@pandas_udf(StringType())
def rm_space_in_quote_udf(x: pd.Series) -> pd.Series:
    def rm_space_in_quote(string):
        encoded_string = string.encode('utf-8')
        decoded_string = encoded_string.decode('utf-8')
        no_space_string = re.sub(r"'[^']*'", lambda m: m.group().replace(" ", ""), decoded_string)
        return no_space_string

    return x.apply(rm_space_in_quote)

df = origin_df.withColumn("content", rm_space_in_quote_udf(origin_df["content"]))\

user_dict= {"뉴진스","트와이스"}


# 사용자 정의사전 만들기
@pandas_udf(ArrayType(StringType()))
def make_user_dict_udf(x: pd.Series) -> pd.Series:
    def make_user_dict(string):
        encoded_string = string.encode('utf-8')
        decoded_string = encoded_string.decode('utf-8')
        substrings = re.findall(r"'(.*?)'", decoded_string)
        return substrings
    return x.apply(make_user_dict)

df_dict = df.select("content")
df_dict= df_dict.withColumn("content", make_user_dict_udf(df["content"]))
# df_dict.collect()
for w in df_dict.toLocalIterator():
    user_dict.update(w["content"])

pecab = PeCab(user_dict=user_dict)

@pandas_udf(StringType())
def clean_str_udf(x: pd.Series) -> pd.Series:
    def clean_str(x):
        punc = string.punctuation
        for ch in punc:
            x = x.replace(ch, '')
        return x
    return x.apply(clean_str)

df = df.withColumn("content", clean_str_udf(df["content"])) \
        .withColumn("title", clean_str_udf(df["title"])) \
        .withColumn("summary", clean_str_udf(df["summary"])) 

 df.groupBy("content").agg(sum(df.content+df.summary+ df.title)).show()

string_per_news = []
df_collector = df.toLocalIterator()
for row in df_collector:
    string_per_news.append(row["content"]+ " " + row["title"] + " " + row["summary"])

def set_words(words):
    counter = Counter(words)
    return counter.most_common(10)

    
words_rdd = sc.parallelize(string_per_news)
words_rdd = words_rdd.map(lambda x: pecab.nouns(x))

words_rdd = words_rdd.map(lambda x: set_words(x))
words_rdd.collect()



In [21]:
# origin_df.select("idol").collect()
# spark.createDataFrame(words_rdd).collect()
import pyspark.sql.functions as F
words_df = df.withColumn("total_content", F.concat(F.col("content"), F.col("summary"), F.col("title")))\
    .select(F.col("idol"),F.col("total_content"))
df.withCoulmn


[Row(idol='TWICE', total_content='걸그룹 트와이스가 K팝 여가수 최초로 빌보드위민인뮤직에서 수상했습니다 소속사 JYP엔터테인먼트는 걸그룹 트와이스가 현지시간으로 지난 1일 미국 로스앤젤레스 유튜브 시어터에서 열린 빌보드위민인뮤직 시상식에 참석해 브레이크스루아티스트 부문에서 수상했다고 밝혔습니다 브레이크스루아티스트 부문은 음악 시장에서 의미있는 도전을 해내고 두각을 드러낸 가수에게 주는 상입니다빌보드가 주최하는 이 시상식은 한 해 음악 산업에 큰 영향을 끼친 여성 아티스트와 크리에이터 프로듀서 등에 시상합니다신새롬 기자 romiynacokr연합뉴스TV 기사문의 및 제보  카톡라인 jebo23 걸그룹 트와이스가 K팝 여가수 최초로 빌보드위민인뮤직에서 수상했습니다소속사 JYP엔터테인먼트는 걸그룹 트와이스가 현지시간으로 지난 1일 미국 로스앤젤레스 유튜브 시어터에서 열린 빌보드위민인뮤직트와이스 K팝 여가수 최초 빌보드위민인뮤직 수상'),
 Row(idol='NewJeans', total_content=' 뉴진스에게 이리 애기애기하던 시절이 있었다니……이 영상 박제해요민희진 대표가 자신의 개인 계정에 뉴진스의 민지 혜인의 데뷔 전 영상을 올려 눈길을 끈다영상 속 민지와 혜인은 지금보다 훨씬 앳되보이는 모습 냉장고의 글자 자석을 이용해 뉴진스를 만드는 표정이 모든 것이 신기하고 의욕에 넘치는 시절을 보여주는 듯 생기가 넘친다이들은 두분한번일어나주시죠란 말에 냉장고 앞에 앉아있다가 벌떡 일어나 청바지 뒤가 보이게 포즈를 취했다 청바지엔 각기 다른 모양으로 뉴진스 레터링이 장식되어 있어 귀여움을 더했다한편 민희진 어도어ADOR 대표는 2023빌보드우먼인뮤직2023 BILLBOARD WOMEN IN MUSIC에 선정됐다23일현지시간 빌보드는 공식 홈페이지를 통해 빌보드우먼인뮤직 리스트를 발표했다 이 가운데 리스트에는 민희진 어도어 대표도 언급됐다빌보드 우먼 인 뮤직은 한 해 동안 음악 산업에 큰 영향을 끼친 여성 아티스트 크리에이터 프로듀서 경영진 

In [1]:
%%time 
import pyspark
import string
from pecab import PeCab
import pandas as pd
import numpy as np
from collections import Counter
from pyspark.sql.functions import pandas_udf, monotonically_increasing_id, regexp_replace, sum, asc, \
                                   arrays_zip, col, explode, sort_array, struct, lit, PandasUDFType, coalesce 
from pyspark.sql.types import StructType, StructField, StringType, MapType, ArrayType
import re
import subprocess


# local[*] : 모든 코어를 사용하겠다, local[4] : 4개의 코어를 사용하겠다.

spark = SparkSession.builder\
        .master("local[3]")\
        .appName("WordCount")\
        .config("spark.driver.memory", "2g")\
        .getOrCreate()
sc=spark.sparkContext

CPU times: user 312 ms, sys: 118 ms, total: 430 ms
Wall time: 1.71 s


In [5]:
# 아이돌 별 뉴스 기사 수

data_name= "test_csv"
df = spark.read.options(header='True', inferSchema='True', delimiter='|',quote='"', multiLine="true").option("encoding", "UTF-8").csv(f"file:///home/j8a507/cluster/news/{data_name}.csv")

df.groupBy("idol").count().write.csv("idol_news_count")

# Replace "/path/to/partitioned/data" with the path to your partitioned data in HDFS
hdfs_path = "idol_news_count"

# Replace "/path/to/local/directory" with the path to your local directory
local_path = "file:///home/j8a507/cluster/news/idol_news_count"

# Download the partitioned directory from HDFS to your local machine
subprocess.check_call(["hdfs", "dfs", "-getmerge", hdfs_path, local_path])

0

In [ ]:

df_total = spark.read.options(header='True', inferSchema='True').option("encoding", "UTF-8").csv(f"file:///home/j8a507/cluster/news/output.csv")

In [12]:
# define schema
schema = StructType([
    StructField("keywords", StringType(), True),
    StructField("index", IntegerType(), True)
])

# create sample data
data = ([('a', 5),('b',4)], 0), ([('c', 3),('d',2)], 1)

# convert to PySpark DataFrame
rdd = spark.sparkContext.parallelize(data)
df = spark.createDataFrame(rdd, schema)

# explode the list of tuples
df = df.withColumn("keywords", explode(transform(col("keywords"), lambda x: struct(*x))))

# count the frequency of each keyword
df = df.groupBy("keywords").count()

# sort by count in descending order
df = df.orderBy("count", ascending=False)

# show result
df.show()

TypeError: Column is not iterable